1️⃣ <b>[FOLDS] <u>Cross-Validation in Time Series  </u></b>

Starting from this single Time Series:
- We will create <font color="#c91ac9">**FOLDS**</font>
- <font color=blue>**Train**</font>/<font color="#ff8005">**Evaluate**</font> our LSTM  <font color="#c91ac9">**on each of these different FOLDS**</font> to conclude about <b><u>the robustness of the model</u><b>.
    
_It is very common to create hundreds of folds in Time Series forecasting, in order to cover all types of external conditions: crash market periods, bull markets, atone markets, etc..._

In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

In [9]:
# Load data
df = pd.read_csv('../raw_data/cleaned_merge_df_top10.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# Assuming 'sales' is the target column
data = df['sales'].values.reshape(-1, 1)

# Scaling the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# Function to create dataset for LSTM
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 28
X, y = create_dataset(data_scaled, look_back)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Define the number of splits for cross-validation
n_splits = 10
tscv = TimeSeriesSplit(n_splits=n_splits)

# Lists to store scores for each fold
mae_scores = []

# Time-series cross-validation
for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Build LSTM model
    model = Sequential([
        LSTM(50, input_shape=(look_back, 1)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

    # Train the model
    model.fit(X_train, y_train, epochs=20, batch_size=1, verbose=0)

    # Making predictions
    predictions = model.predict(X_test)
    predictions_rescaled = scaler.inverse_transform(predictions)
    y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Calculate MAE
    mae = mean_absolute_error(y_test_rescaled, predictions_rescaled)
    mae_scores.append(mae)
    print(f"Fold MAE: {mae}")

# Print overall results
average_mae = np.mean(mae_scores)
print(f"Average MAE across all folds: {average_mae}")


55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Fold MAE: 19.471964677907355
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Fold MAE: 14.963129867331773
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Fold MAE: 15.101673601856154
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Fold MAE: 13.85513115918032
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Fold MAE: 14.218155355660528
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Fold MAE: 11.183748382946245
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Fold MAE: 11.4414965276336
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Fold MAE: 10.97659278801784
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Fold MAE: 10.04035466421859
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Fold MAE: 9.8857844714247
Average MAE across all folds: 13.11380314961771


2️⃣ <b>[TRAIN-TEST SPLIT] <u>Holdout method</u></b>

For each <font color="#c91ac9">**FOLD**</font>, we will do a <font color=blue>**TRAIN**</font>-<font color="#ff8005">**TEST**</font> SPLIT to:
* <font color=blue>**fit**</font> the model on the <font color=blue>**train**</font> set
* <font color="#ff8005">**evaluate**</font> it on the <font color="#ff8005">**test**</font> set

_Always split the train set **chronologically** before the test set!_

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

In [7]:
# Load data
df = pd.read_csv('../raw_data/cleaned_merge_df_top10.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# Scaling the 'sales' data to be between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['sales']])

# Function to create dataset for LSTM
def create_dataset(data, look_back=1):
    X, Y = [], []
    for i in range(len(data) - look_back):
        a = data[i:(i + look_back), 0]
        X.append(a)
        Y.append(data[i + look_back, 0])
    return np.array(X), np.array(Y)

# Define the number of previous days to use for predicting the next day
look_back = 28
X, y = create_dataset(scaled_data, look_back)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split data into train and test sets
train_size = int(len(X) * 0.80)  # 80% for training
test_size = len(X) - train_size
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build LSTM model
model = Sequential([
    LSTM(50, input_shape=(look_back, 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=1, verbose=1)

# Making predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Invert predictions to original scale
train_predict = scaler.inverse_transform(train_predict)
y_train_inv = scaler.inverse_transform([y_train])
test_predict = scaler.inverse_transform(test_predict)
y_test_inv = scaler.inverse_transform([y_test])

# Calculate and print RMSE for evaluation
train_score = np.sqrt(mean_squared_error(y_train_inv[0], train_predict[:,0]))
print(f'Train Score: {train_score:.2f} RMSE')
test_score = np.sqrt(mean_squared_error(y_test_inv[0], test_predict[:,0]))
print(f'Test Score: {test_score:.2f} RMSE')

# Predict the next 28 days
last_batch = scaled_data[-look_back:]
last_batch = last_batch.reshape((1, look_back, 1))
next_28_days = model.predict(last_batch)
next_28_days = scaler.inverse_transform(next_28_days)
print("Sales forecast for the next 28 days:", next_28_days.flatten())


Epoch 1/20
15281/15281 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - loss: 0.0025
Epoch 2/20
15281/15281 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - loss: 0.0020
Epoch 3/20
15281/15281 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - loss: 0.0017
Epoch 4/20
15281/15281 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - loss: 0.0019
Epoch 5/20
15281/15281 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - loss: 0.0017
Epoch 6/20
15281/15281 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - loss: 0.0012
Epoch 7/20
15281/15281 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - loss: 0.0011
Epoch 8/20
15281/15281 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - loss: 0.0012
Epoch 9/20
15281/15281 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - loss: 0.0011
Epoch 10/20
15281/15281 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - loss: 0.0011
Epoch 11/20
15281/15281 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - loss: 0.0010
Epoch 12/20
15281/15281 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - loss: 0.0010
Epoch 13/20
15281/15281 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - loss: 9.6573e-04
Epoch 14/20
15281/15281 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/ste